In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import numpy as np
import os

In [2]:
tf.__version__

'2.0.0'

In [3]:
tf.test.is_gpu_available()

False

In [4]:
train_path=glob.glob('/Users/mac/PycharmProjects/dc_2000/train/*/*.jpg')
test_path=glob.glob('/Users/mac/PycharmProjects/dc_2000/test/*/*.jpg')

In [5]:
train_count=len(train_path)
test_count=len(test_path)

In [6]:
train_count, test_count

(2000, 1000)

In [7]:
train_path[:5]

['/Users/mac/PycharmProjects/dc_2000/train/cat/cat.952.jpg',
 '/Users/mac/PycharmProjects/dc_2000/train/cat/cat.946.jpg',
 '/Users/mac/PycharmProjects/dc_2000/train/cat/cat.6.jpg',
 '/Users/mac/PycharmProjects/dc_2000/train/cat/cat.749.jpg',
 '/Users/mac/PycharmProjects/dc_2000/train/cat/cat.991.jpg']

In [8]:
p = '/Users/mac/PycharmProjects/dc_2000/train/cat/cat.952.jpg'

In [9]:
int(p.split('/')[-2] == 'cat')

1

In [10]:
train_label = [int(path.split('/')[-2] == 'cat') for path in train_path]
test_label = [int(path.split('/')[-2] == 'cat') for path in test_path]

In [11]:
train_label[:5]

[1, 1, 1, 1, 1]

In [12]:
def image_preprocess(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [256,256])
    image = tf.cast(image, tf.float32)
    image = image/255
    return image, label

In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_path, train_label))
test_dataset = tf.data.Dataset.from_tensor_slices((test_path, test_label))

In [14]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [15]:
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)

In [16]:
BATCH_SIZE=32

In [17]:
train_dataset = train_dataset.shuffle(train_count).repeat().batch(BATCH_SIZE)
test_dataset = test_dataset.repeat().batch(BATCH_SIZE)

In [18]:
train_dataset, test_dataset

(<BatchDataset shapes: ((None, 256, 256, 3), (None,)), types: (tf.float32, tf.int32)>,
 <BatchDataset shapes: ((None, 256, 256, 3), (None,)), types: (tf.float32, tf.int32)>)

In [19]:
# for img, label in test_dataset.take(1):
#     plt.show(img)

In [20]:
conv_base = tf.keras.applications.VGG16(weights='imagenet', include_top=False)

In [21]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [22]:
model = tf.keras.Sequential()
model.add(conv_base)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, None, None, 512)   14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 14,977,857
Trainable params: 14,977,857
Non-trainable params: 0
_________________________________________________________________


In [24]:
conv_base.trainable=False

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, None, None, 512)   14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 14,977,857
Trainable params: 263,169
Non-trainable params: 14,714,688
_________________________________________________________________


In [26]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='binary_crossentropy', metrics=['acc'])

In [29]:
history = model.fit(
    train_dataset, 
    steps_per_epoch=train_count//BATCH_SIZE, 
    epochs=15, 
    validation_data=test_dataset,
    validation_steps=test_count//BATCH_SIZE
)

Train for 62 steps, validate for 31 steps
Epoch 1/15
17/62 [=======>......................] - ETA: 6:17 - loss: 0.6815 - acc: 0.5879

KeyboardInterrupt: 

In [27]:
conv_base.trainable = True

In [31]:
len(conv_base.layers)

19

In [29]:
fine_tune_at = -3

In [32]:
for layer in conv_base.layers[:fine_tune_at]:
    layer.trainable = False

In [33]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005/10),loss='binary_crossentropy', metrics=['acc'])

In [34]:
initial_epoch = 12
fine_tune_epoch = 10
total_epochs = initial_epoch + fine_tune_epoch

In [35]:
history = model.fit(
    train_dataset,
    steps_per_epoch=train_count//BATCH_SIZE,
    initial_epoch=initial_epoch,
    validation_data=test_dataset,
    validation_steps=test_count//BATCH_SIZE
)

Train for 62 steps, validate for 31 steps
